In [1]:
# Ensure all necessary imports are present
from transformers import ViTFeatureExtractor
import numpy as np
from PIL import Image
from datasets import load_dataset
from datasets import DatasetDict
import torch

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load dataset
ds = load_dataset('betul2')
ds = ds.rename_column('label', 'labels')

# Split the dataset
ds = ds['train'].train_test_split(test_size=0.1)
train_val = ds['train'].train_test_split(test_size=0.1)
ds['train'] = train_val['train']
ds['validation'] = train_val['test']
ds['test'] = ds['test']

# Convert to DatasetDict
dataset = DatasetDict(ds)


Using the latest cached version of the dataset since betul2 couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at C:\Users\semih\.cache\huggingface\datasets\betul2\default\0.0.0\c8b151b1a7bf91b4 (last modified on Thu Aug 22 16:40:50 2024).


In [3]:
# Feature extractor initialization
model_name_or_path = 'google/vit-base-patch16-224-in21k'
feature_extractor = ViTFeatureExtractor.from_pretrained(model_name_or_path)

C:\Users\semih\AppData\Roaming\Python\Python312\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [4]:
# Function to process a single example
def process_example(example):
    image = example['image']
    # Convert the PIL image to RGB format
    image = image.convert('RGB')
    # Convert the PIL image to numpy array
    image_array = np.array(image)
    # Ensure the image is in (height, width, channels) format and rearrange to (channels, height, width)
    if image_array.ndim == 3 and image_array.shape[-1] in [1, 3, 4]:  # last dimension is channels
        image_array = np.moveaxis(image_array, -1, 0)
    # Now the image should be in (channels, height, width) format
    inputs = feature_extractor(image_array, return_tensors='pt')
    inputs['labels'] = example['labels']
    return inputs


In [5]:
# Test the process_example function
processed_example = process_example(ds['train'][0])
print(processed_example)

{'pixel_values': tensor([[[[ 0.0039,  0.0039,  0.0353,  ...,  0.0039, -0.0431,  0.0353],
          [ 0.0039,  0.0039,  0.0353,  ...,  0.0039, -0.0431,  0.0353],
          [ 0.0039,  0.0039,  0.0039,  ...,  0.0039, -0.0431,  0.0431],
          ...,
          [ 0.0039,  0.0039,  0.0039,  ...,  0.0039,  0.0039,  0.0039],
          [-0.0275, -0.0275,  0.0039,  ...,  0.0039,  0.0039,  0.0039],
          [-0.0196, -0.0275,  0.0039,  ...,  0.0039,  0.0039,  0.0039]],

         [[ 0.0039,  0.0039,  0.0353,  ...,  0.0039, -0.0118,  0.0353],
          [ 0.0039,  0.0039,  0.0353,  ...,  0.0039, -0.0118,  0.0353],
          [ 0.0039,  0.0039,  0.0039,  ...,  0.0039, -0.0118,  0.0431],
          ...,
          [ 0.0039,  0.0039,  0.0039,  ...,  0.0039,  0.0039,  0.0039],
          [-0.0275, -0.0275,  0.0039,  ...,  0.0039,  0.0039,  0.0039],
          [-0.0196, -0.0275,  0.0039,  ...,  0.0039,  0.0039,  0.0039]],

         [[ 0.0039,  0.0039,  0.0039,  ...,  0.0039,  0.0039,  0.0039],
          [ 0

In [6]:
# Function to transform a batch of examples
def transform(example_batch):
    images = [x.convert('RGB') for x in example_batch['image']]
    # Ensure each image is in the correct format
    images = [np.moveaxis(np.array(img), -1, 0) if img.ndim == 3 and img.shape[-1] in [1, 3, 4] else np.array(img) for img in images]
    inputs = feature_extractor(images, return_tensors='pt')
    inputs['labels'] = example_batch['labels']
    return inputs

In [7]:
# Function to transform a batch of examples
def transform(example_batch):
    images = [x.convert('RGB') for x in example_batch['image']]
    # Ensure each image is in the correct format
    images = [np.moveaxis(np.array(img), -1, 0) if np.array(img).ndim == 3 and np.array(img).shape[-1] in [1, 3, 4] else np.array(img) for img in images]
    inputs = feature_extractor(images, return_tensors='pt')
    inputs['labels'] = example_batch['labels']
    return inputs

In [8]:
# Apply the transform function to the dataset
prepared_ds = ds.with_transform(transform)

In [9]:
def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['labels'] for x in batch])
    }

In [10]:
# Define evaluation metric
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy", trust_remote_code=True)
def compute_metrics(p):
    return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)

C:\Users\semih\AppData\Local\Temp\ipykernel_17884\267560914.py:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy", trust_remote_code=True)


In [11]:
# Load pretrained model
from transformers import ViTForImageClassification

labels = ds['train'].features['labels'].names
model = ViTForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=len(labels),
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)}
)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
# Move the model to the GPU
model.to(device)

ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTSdpaAttention(
            (attention): ViTSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_fe

In [13]:
# Define training arguments
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="./vit-base--v5",
  per_device_train_batch_size=16,
  evaluation_strategy="steps",
  num_train_epochs=4,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=2e-4,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='tensorboard',
  load_best_model_at_end=True,
)

C:\Users\semih\AppData\Roaming\Python\Python312\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [14]:
# Initialize Trainer
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=prepared_ds["train"],
    eval_dataset=prepared_ds["validation"],
    tokenizer=feature_extractor,
)

In [15]:
import torch

# Train the model
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

# Evaluate the model
metrics = trainer.evaluate(prepared_ds['test'])
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

# Push to hub or create model card
kwargs = {
    "finetuned_from": model.config._name_or_path,
    "tasks": "image-classification",
    "dataset": 'custom brats layers',
    "tags": ['image-classification'],
}

if training_args.push_to_hub:
    trainer.push_to_hub('🍻 cheers', **kwargs)
else:
    trainer.create_model_card(**kwargs)

  0%|          | 0/744 [00:00<?, ?it/s]

C:\Users\semih\AppData\Roaming\Python\Python312\site-packages\transformers\models\vit\modeling_vit.py:252: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  context_layer = torch.nn.functional.scaled_dot_product_attention(


{'loss': 1.1088, 'grad_norm': 1.39084792137146, 'learning_rate': 0.00019731182795698925, 'epoch': 0.05}
{'loss': 0.8508, 'grad_norm': 1.115374207496643, 'learning_rate': 0.0001946236559139785, 'epoch': 0.11}
{'loss': 0.8677, 'grad_norm': 1.134900450706482, 'learning_rate': 0.00019193548387096775, 'epoch': 0.16}
{'loss': 0.8366, 'grad_norm': 1.3971041440963745, 'learning_rate': 0.000189247311827957, 'epoch': 0.22}
{'loss': 0.7954, 'grad_norm': 1.8755857944488525, 'learning_rate': 0.00018655913978494625, 'epoch': 0.27}
{'loss': 0.7438, 'grad_norm': 2.1053378582000732, 'learning_rate': 0.00018387096774193548, 'epoch': 0.32}
{'loss': 0.8199, 'grad_norm': 1.3887600898742676, 'learning_rate': 0.00018118279569892475, 'epoch': 0.38}
{'loss': 0.8945, 'grad_norm': 5.668015956878662, 'learning_rate': 0.00017849462365591398, 'epoch': 0.43}
{'loss': 0.673, 'grad_norm': 1.4448820352554321, 'learning_rate': 0.00017580645161290325, 'epoch': 0.48}
{'loss': 0.7009, 'grad_norm': 2.706613063812256, 'learn

  0%|          | 0/42 [00:00<?, ?it/s]

{'eval_loss': 0.7025018930435181, 'eval_accuracy': 0.7636363636363637, 'eval_runtime': 2.2207, 'eval_samples_per_second': 148.604, 'eval_steps_per_second': 18.913, 'epoch': 0.54}
{'loss': 0.6874, 'grad_norm': 1.2416284084320068, 'learning_rate': 0.00017043010752688172, 'epoch': 0.59}
{'loss': 0.7256, 'grad_norm': 1.7809841632843018, 'learning_rate': 0.00016774193548387098, 'epoch': 0.65}
{'loss': 0.7037, 'grad_norm': 0.9938160181045532, 'learning_rate': 0.00016505376344086022, 'epoch': 0.7}
{'loss': 0.6592, 'grad_norm': 0.8089711666107178, 'learning_rate': 0.00016236559139784946, 'epoch': 0.75}
{'loss': 0.7134, 'grad_norm': 0.8656672835350037, 'learning_rate': 0.00015967741935483872, 'epoch': 0.81}
{'loss': 0.7579, 'grad_norm': 0.8881009817123413, 'learning_rate': 0.00015698924731182796, 'epoch': 0.86}
{'loss': 0.6383, 'grad_norm': 3.4207422733306885, 'learning_rate': 0.00015430107526881722, 'epoch': 0.91}
{'loss': 0.7042, 'grad_norm': 1.1370435953140259, 'learning_rate': 0.00015161290

...

{'eval_loss': 0.5405390858650208, 'eval_accuracy': 0.793939393939394, 'eval_runtime': 1.9088, 'eval_samples_per_second': 172.887, 'eval_steps_per_second': 22.004, 'epoch': 3.23}
{'loss': 0.2442, 'grad_norm': 1.3763409852981567, 'learning_rate': 3.602150537634409e-05, 'epoch': 3.28}
{'loss': 0.3434, 'grad_norm': 4.556057929992676, 'learning_rate': 3.3333333333333335e-05, 'epoch': 3.33}
{'loss': 0.4097, 'grad_norm': 0.828928530216217, 'learning_rate': 3.0645161290322585e-05, 'epoch': 3.39}
{'loss': 0.3498, 'grad_norm': 3.557579755783081, 'learning_rate': 2.7956989247311828e-05, 'epoch': 3.44}
{'loss': 0.2932, 'grad_norm': 2.2648048400878906, 'learning_rate': 2.5268817204301075e-05, 'epoch': 3.49}
{'loss': 0.2564, 'grad_norm': 2.0926642417907715, 'learning_rate': 2.258064516129032e-05, 'epoch': 3.55}
{'loss': 0.2702, 'grad_norm': 3.5984246730804443, 'learning_rate': 1.989247311827957e-05, 'epoch': 3.6}
{'loss': 0.2812, 'grad_norm': 0.8931816816329956, 'learning_rate': 1.7204301075268818e-

  0%|          | 0/42 [00:00<?, ?it/s]

{'eval_loss': 0.5512449145317078, 'eval_accuracy': 0.8, 'eval_runtime': 1.8804, 'eval_samples_per_second': 175.491, 'eval_steps_per_second': 22.335, 'epoch': 3.76}
{'loss': 0.3536, 'grad_norm': 1.8082451820373535, 'learning_rate': 9.13978494623656e-06, 'epoch': 3.82}
{'loss': 0.2789, 'grad_norm': 0.4126892685890198, 'learning_rate': 6.451612903225806e-06, 'epoch': 3.87}
{'loss': 0.2241, 'grad_norm': 0.8909844756126404, 'learning_rate': 3.763440860215054e-06, 'epoch': 3.92}
{'loss': 0.2632, 'grad_norm': 2.3688242435455322, 'learning_rate': 1.0752688172043011e-06, 'epoch': 3.98}
{'train_runtime': 163.7833, 'train_samples_per_second': 72.413, 'train_steps_per_second': 4.543, 'train_loss': 0.5439459455590094, 'epoch': 4.0}
***** train metrics *****
  epoch                    =         4.0
  total_flos               = 855959680GF
  train_loss               =      0.5439
  train_runtime            =  0:02:43.78
  train_samples_per_second =      72.413
  train_steps_per_second   =       4.543

  0%|          | 0/46 [00:00<?, ?it/s]

***** eval metrics *****
  epoch                   =        4.0
  eval_accuracy           =     0.7902
  eval_loss               =     0.5956
  eval_runtime            = 0:00:02.43
  eval_samples_per_second =    150.539
  eval_steps_per_second   =     18.869
